# 01. Import libraries

In [1]:
# import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Create a path for easier importing
path = r'C:\Users\blim9\Desktop\Projects'

In [3]:
# Import the data set
df = pd.read_csv(os.path.join (path, 'Amazon Prime Video', 'Data', 'amazon_prime_titles.csv'), index_col = False)

In [4]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7586 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
dtypes: int64(1), object(11)
memory usage: 906.5+ KB


#### There are many null values for both country and date_added. This is something to be aware of when conducting further analysis.

# 02. Data Wrangling

In [6]:
# Check the types of content Prime Video offers
df.type.value_counts()

Movie      7814
TV Show    1854
Name: type, dtype: int64

In [7]:
# Check the ratings of the content
df.rating.value_counts()

13+         2117
16+         1547
ALL         1268
18+         1243
R           1010
PG-13        393
7+           385
PG           253
NR           223
TV-14        208
TV-PG        169
TV-NR        105
G             93
TV-G          81
TV-MA         77
TV-Y          74
TV-Y7         39
UNRATED       33
NC-17          3
AGES_18_       3
NOT_RATE       3
AGES_16_       2
16             1
ALL_AGES       1
Name: rating, dtype: int64

#### There are many ratings, but some overlap or are just named differently. 
#### https://www.primevideo.com/help/ref=atv_hp_nd_cnt?nodeId=GFGQU3WYEG6FSJFJ was used as reference.

In [8]:
# Rename some ratings to ensure consistency
df['rating'].replace({"NC-17": "18+", "AGES_18_": "18+", "NOT_RATE": "NR", "AGES_16_": "16+", "16": "16+", "ALL_AGES": 
                     "ALL"}, inplace=True)

In [9]:
df.rating.value_counts()

13+        2117
16+        1550
ALL        1269
18+        1249
R          1010
PG-13       393
7+          385
PG          253
NR          226
TV-14       208
TV-PG       169
TV-NR       105
G            93
TV-G         81
TV-MA        77
TV-Y         74
TV-Y7        39
UNRATED      33
Name: rating, dtype: int64

In [10]:
# Check for missing values
df.isnull().sum()

show_id            0
type               0
title              0
director        2082
cast            1233
country         8996
date_added      9513
release_year       0
rating           337
duration           0
listed_in          0
description        0
dtype: int64

#### There are too many null values for date_added, so I will omit it from the analysis since it won't help. The director and cast columns are not too relevant to me either, so I will drop them.

#### I will also drop the 337 missing values for the ratings.

In [11]:
# Drop the director and cast columns
df = df.drop(columns = ['director', 'cast', 'date_added'])

In [12]:
df.isnull().sum()

show_id            0
type               0
title              0
country         8996
release_year       0
rating           337
duration           0
listed_in          0
description        0
dtype: int64

In [13]:
df.dropna(subset = ['rating'], inplace=True)

In [14]:
df.isnull().sum()

show_id            0
type               0
title              0
country         8674
release_year       0
rating             0
duration           0
listed_in          0
description        0
dtype: int64

#### Though country has many null values, I may sitll find importance in its results so I will not drop it.

In [15]:
# Create a new dataframe for genre visualization
df_split = df.copy()

In [16]:
df_split = pd.concat([df_split, df["listed_in"].str.split(",", expand = True)], axis = 1)
df_split

,show_id,type,title,country,release_year,rating,duration,listed_in,description,0,1,2,3,4
1,s2,Movie,Take Care Good Night,India,2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,Drama,International,None,None,None
16,s17,Movie,Zoombies,NaN,2016,13+,87 min,"Horror, Science Fiction",When a strange virus quickly spreads through a...,Horror,Science Fiction,None,None,None
17,s18,TV Show,Zoo Babies,NaN,2008,ALL,1 Season,"Kids, Special Interest",A heart warming and inspiring series that welc...,Kids,Special Interest,None,None,None
18,s19,TV Show,Zoë Coombs Marr: Bossy Bottom,NaN,2020,18+,1 Season,"Comedy, Talk Show and Variety",Zoë Coombs Marr has been on hiatus. Sort of. F...,Comedy,Talk Show and Variety,None,None,None
19,s20,Movie,Zoe,NaN,2018,R,104 min,Science Fiction,ZOE tells a tale of forbidden love between an ...,Science Fiction,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9663,s9664,Movie,Pride Of The Bowery,NaN,1940,7+,60 min,Comedy,New York City street principles get an East Si...,Comedy,None,None,None,None
9664,s9665,TV Show,Planet Patrol,NaN,2018,13+,4 Seasons,TV Shows,"This is Earth, 2100AD - and these are the adve...",TV Shows,None,None,None,None
9665,s9666,Movie,Outpost,NaN,2008,R,90 min,Action,"In war-torn Eastern Europe, a world-weary grou...",Action,None,None,None,None
9666,s9667,TV Show,Maradona: Blessed Dream,NaN,2021,TV-MA,1 Season,"Drama, Sports","The series tells the story of Diego Maradona, ...",Drama,Sports,None,None,None


In [17]:
df_split = df_split.melt(id_vars = ["type", "title"], value_vars = range(4), value_name = "genre")
df_split = df_split[df_split["genre"].notna()]
df_split

,type,title,variable,genre
0,Movie,Take Care Good Night,0,Drama
1,Movie,Zoombies,0,Horror
2,TV Show,Zoo Babies,0,Kids
3,TV Show,Zoë Coombs Marr: Bossy Bottom,0,Comedy
4,Movie,Zoe,0,Science Fiction
...,...,...,...,...
37201,Movie,Justin Bieber: Our World,3,Documentary
37276,Movie,The Rocky Horror Picture Show,3,Comedy
37277,Movie,The Rocker,3,Comedy
37286,Movie,Snow White,3,Kids


In [18]:
df_split.genre.value_counts()

Drama                         2152
Action                        1615
Comedy                        1433
 Drama                        1419
 Suspense                     1244
Documentary                    833
 Kids                          684
 Comedy                        565
 Special Interest              534
 Romance                       526
Horror                         519
Animation                      487
 Entertainment                 431
 and Culture                   429
Arts                           405
Kids                           367
 Science Fiction               357
 Horror                        324
 International                 301
TV Shows                       262
Suspense                       189
 Adventure                     186
Special Interest               181
 Sports                        135
Arthouse                       132
 Western                       126
Romance                        125
 Unscripted                    119
Music Videos and Con

In [19]:
df_split['genre'] = df_split['genre'].str.strip()

In [20]:
# There is a genre called "Arts, Entertainment, and Culture" which got split into 3
# Replace them back into one column called "Arts, Entertainment, and Culture"
df_split['genre'].replace({"Arts": "Arts, Entertainment, and Culture", "Entertainment": "Arts, Entertainment, and Culture",
                                      "and Culture": "Arts, Entertainment, and Culture"}, inplace=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9331 entries, 1 to 9667
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9331 non-null   object
 1   type          9331 non-null   object
 2   title         9331 non-null   object
 3   country       657 non-null    object
 4   release_year  9331 non-null   int64 
 5   rating        9331 non-null   object
 6   duration      9331 non-null   object
 7   listed_in     9331 non-null   object
 8   description   9331 non-null   object
dtypes: int64(1), object(8)
memory usage: 729.0+ KB


In [22]:
# Change the datatype for the release_year to int16
df.release_year = df.release_year.astype('int16')

In [23]:
df.head()

,show_id,type,title,country,release_year,rating,duration,listed_in,description
1,s2,Movie,Take Care Good Night,India,2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
16,s17,Movie,Zoombies,NaN,2016,13+,87 min,"Horror, Science Fiction",When a strange virus quickly spreads through a...
17,s18,TV Show,Zoo Babies,NaN,2008,ALL,1 Season,"Kids, Special Interest",A heart warming and inspiring series that welc...
18,s19,TV Show,Zoë Coombs Marr: Bossy Bottom,NaN,2020,18+,1 Season,"Comedy, Talk Show and Variety",Zoë Coombs Marr has been on hiatus. Sort of. F...
19,s20,Movie,Zoe,NaN,2018,R,104 min,Science Fiction,ZOE tells a tale of forbidden love between an ...


# 03. Data consistency check

In [24]:
df.isnull().sum()

show_id            0
type               0
title              0
country         8674
release_year       0
rating             0
duration           0
listed_in          0
description        0
dtype: int64

In [25]:
# Check for duplicate values
df_dups = df[df.duplicated()]

In [26]:
df_dups

,show_id,type,title,country,release_year,rating,duration,listed_in,description


### No duplicates

In [27]:
# Export the cleaned data set for visualization
df.to_csv(os.path.join (path, 'Amazon Prime Video', 'Data', 'cleaned_prime.csv'))

In [28]:
df_split.to_csv(os.path.join (path, 'Amazon Prime Video', 'Data', 'split_prime.csv'))